## 初期設定

In [ ]:
# --- Notebook初期設定 ---
%load_ext autoreload
%autoreload 2
import src.config as cfg
import src.data_loader as dl
import src.data_cleaning_utils as cu
print("🔁 autoreload 有効化完了")
import yaml
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from collections import defaultdict

In [ ]:
# 設定読み込み
settings = cfg.load_settings("setting.yaml")
display(settings)

## 本ファイルの説明
データが最低限の品質基準を満たしているかを網羅的に確認し、ファイルロードからクリーニングまでのプロセスを決定します。

<details>
<summary><b>結果</b></summary>


</details>


### データのロードの確認
ファイル重複やファイル欠損、破壊がないかを確かめ、プロセスを確定します。
<details>
<summary><b>結果</b></summary>

ファイルの重複や欠損はなく、正しくロードされました。  
修正処理は不要です。
</details>

In [ ]:
print("\nファイル毎にデータをロードします")
df_DATAs_BY_ALL_FILEs = {
    (filename, year): dl.load_on_startup(settings["data_path"]["raw"], str(year), filename,header=1)
    for filename in settings["files"]["raw"]
    for year in settings["years"]
}
#display(df_DATAs_BY_ALL_FILEs)

print("\n次に欠けてるファイルがないか確認します")
if all(dl.chk_file_missing(df) for df in df_DATAs_BY_ALL_FILEs.values()):
    print("✅ ファイル欠損はありませんでした。")

print("\n重複ファイルがないか確認します")
duplicates = dl.chk_duplicate_dfs(df_DATAs_BY_ALL_FILEs, False,False)
if duplicates:
    print("🔍 重複データフレームが見つかりました：\n")
    for h, key_group in duplicates.items():
        print(f"ハッシュ: {h}")
        for key in key_group:
            print(key)
else:
    print("✅ 重複データフレームはありません。")
print("\nデータロードの確認を終了します。")


### 欠損値表現の確認と処理
欠損値の表現を確認し、修正プロセスを決定します。

<details>
<summary><b>結果</b></summary>

"-"が欠損値として使われています。  
これはsetting.yamlのnan_valuesに登録し欠損値として読み込みます。  
  
また、ゼロも使われています。  
ゼロは値として残し、必要に応じてfeatures_generatorで再計算し更新します。

以下がゼロが使われている項目です。  
{'fy-balance-sheet.csv': ['BPS', '短期借入金', '総資産', '自己資本比率', '長期借入金']  
'fy-cash-flow-statement.csv': ['営業CF', '営業CFマージン', '投資CF', '現金同等物', '設備投資', '財務CF']  
'fy-profit-and-loss.csv': ['EPS', 'ROA', 'ROE', '営業利益', '売上高', '純利益', '経常利益'],  
'fy-stock-dividend.csv': ['一株配当', '剰余金の配当', '純資産配当率', '総還元性向', '自社株買い', '配当性向']}
</details>

In [ ]:
print("\nファイル毎にデータをロードします")
df_DATAs_BY_ALL_FILEs = {
    (filename, year): dl.load_on_startup(settings["data_path"]["raw"], str(year), filename,header=1)
    for filename in settings["files"]["raw"]
    for year in settings["years"]
}
#display(df_DATAs_BY_ALL_FILEs)

print("\nファイルごとに各文字列の個数をカウントします")
df_placeholder_counts = pd.DataFrame()
for (filename,year), df in df_DATAs_BY_ALL_FILEs.items():
    df_tmp = cu.chk_missing_values_expression(df, filename, year)
    df_placeholder_counts = pd.concat([df_placeholder_counts,df_tmp], axis=0)
#display(df_placeholder_counts)

In [22]:
print("/n欠損値と疑われるコードを出力します。")
df = df_placeholder_counts.drop(["ファイル名","オプション値"],axis=1)
df = df.groupby("col").sum().sum()
display(df)

print("")
print("疑われるコードが含まれている列を出力します")
df = df_placeholder_counts.drop(["ファイル名","オプション値"],axis=1)
df = df.groupby("col").sum().sum()
pl_list = df[df >0].index.tolist()
df = df_placeholder_counts.drop(["ファイル名","オプション値"],axis=1)
for value in pl_list:
    list = df[df[value]>0]["col"].unique()
    print("--------------",value,"のチェック----------------------------")
    display(list)

print("")
print("アルファベットが含まれているコードを出力します。")
for (filename, year), df in df_DATAs_BY_ALL_FILEs.items():
    df_tmp = df[df["コード"].str.contains("[A-Za-z]", na=False)]
    code_list = df_tmp["コード"].to_list()
    print(code_list)

print("")
print("ゼロが登録されているファイルと列を出力します。")
df = df_placeholder_counts
df_0 = df.groupby(["ファイル名","col"])["0"].sum()
df_0 = df_0[df_0>0].reset_index()
print(df_0)
print(df_0['col'].dtype)

print("")
print("4つのファイルごとに該当列のリストを準備します")
df_temp = df_0[['ファイル名', 'col']]
grouped = df_temp.groupby('ファイル名')
col_lists = {
    filename: group['col'].tolist()  # 各グループから 'col' 列を抽出し、tolist()でPythonリストに変換
    for filename, group in grouped
}
print(col_lists)

print("")
print("ゼロの含まれるコードのデータを確認します。")
print("まずは年でわかれた各ファイルをつなぎ合わせます")
df_merge = {}
for (filename, year), df in df_DATAs_BY_ALL_FILEs.items():
    if filename not in df_merge:
        df_merge[filename] = []
    df_merge[filename].append(df)
df_final = {}
for filename in df_merge:
    df_final[filename]  = pd.concat(df_merge[filename])
display(df_final["fy-balance-sheet.csv"])

print("")
print("ファイルごとにゼロをもつコードの値をチェックします。")
filename = "fy-profit-and-loss.csv"
print("ファイル名:",filename)
df_all = df_final[filename]
for col_value in col_lists[filename]:
    #print(filename, col)
    df = df_all[df_all[col_value] == "0"]
    code_list = df["コード"].unique()
    print("-----------------------",col_value,"-----------------------")
    for code in code_list:
        display(df_all[df_all["コード"] == code])

/n欠損値と疑われるコードを出力します。


empty            0
space            0
-           167782
―                0
—                0
--               0
Na               0
na               0
N/A              0
n/a              0
None             0
none             0
NULL             0
null             0
0             4155
alphabet      1940
dtype: object


疑われるコードが含まれている列を出力します
-------------- - のチェック----------------------------


array(['総資産', '純資産', '株主資本', '利益剰余金', '短期借入金', '長期借入金', 'BPS', '自己資本比率',
       '営業CF', '投資CF', '財務CF', '設備投資', '営業CFマージン', '現金同等物', '売上高', '営業利益',
       '経常利益', '純利益', 'EPS', 'ROE', 'ROA', '一株配当', '剰余金の配当', '自社株買い',
       '配当性向', '総還元性向', '純資産配当率'], dtype=object)

-------------- 0 のチェック----------------------------


...

----------------------- 経常利益 -----------------------


,コード,年度,売上高,営業利益,経常利益,純利益,EPS,ROE,ROA
1934,6492,2010/11,12649948000,1579759000,1729536000,1018176000,567.86,12.05,8.4
2018,6492,2011/11,10195847000,803624000,939475000,384559000,214.48,4.44,3.45
2063,6492,2012/11,8347631000,338183000,458348000,197148000,109.95,2.27,1.85
2116,6492,2013/11,7404326000,134070000,304043000,187500000,106.12,2.13,1.68
2170,6492,2014/11,7437199000,159981000,351872000,197189000,112.1,2.22,1.83
2230,6492,2015/11,7624708000,373307000,615591000,342001000,196.97,3.76,3.07
2293,6492,2016/11,7501002000,113623000,419623000,226304000,130.36,2.46,2.02
2362,6492,2017/11,8304000000,234430000,373000000,193000000,111.19,2.05,1.37
2437,6492,2018/11,7577000000,-67930000,0,165000000,95.07,1.73,1.19
2497,6492,2019/11,6663000000,-872933000,-792000000,-983000000,-567.26,-,-


,コード,年度,売上高,営業利益,経常利益,純利益,EPS,ROE,ROA
2950,8836,2010/03,2057000000,23000000,49000000,55000000,0.82,1.45,1.09
3054,8836,2011/03,564000000,-98000000,-102000000,-341000000,-5.08,-,-
3123,8836,2012/03,380000000,-89000000,-45000000,-40000000,-0.6,-,-
3210,8836,2013/03,560000000,51000000,92000000,94000000,1.41,2.76,2.28
3307,8836,2014/03,592000000,58000000,73000000,74000000,1.57,2.25,1.89
3411,8836,2015/03,424000000,14000000,26000000,29000000,0.62,0.88,0.75
3500,8836,2016/03,447000000,123000000,115000000,71000000,1.39,2.1,1.79
3595,8836,2017/03,246000000,-162000000,-161000000,-162000000,-1.69,-,-
3677,8836,2018/03,515000000,-138000000,-143000000,179000000,1.86,8.61,7.12
3739,8836,2019/03,141000000,-103000000,-107000000,-109000000,-1.14,-,-


,コード,年度,売上高,営業利益,経常利益,純利益,EPS,ROE,ROA
1656,6029,2010/12,603562000,-,-85121000,-98762000,-17.33,-,-
1721,6029,2011/12,710741000,-,-5655000,-21465000,-3.77,-,-
1761,6029,2012/12,772046000,-24896000,-41981000,-44059000,-7.73,-,-
1812,6029,2013/12,1320152000,144316000,144549000,145542000,25.53,343.79,14.76
1862,6029,2014/12,1490027000,78002000,68673000,27604000,4.76,5.05,1.7
1919,6029,2015/12,2564070000,387415000,453313000,268734000,34.45,32.97,11.21
1981,6029,2016/12,3251564000,293905000,301387000,148703000,18.41,10.06,5.32
2048,6029,2017/12,3759158000,435573000,455475000,301430000,34.59,17.19,8.51
2120,6029,2018/12,3479200000,164280000,163697000,269930000,30.88,13.5,6.01
2180,6029,2019/12,2833781000,25196000,31411000,-146664000,-16.71,-,-


,コード,年度,売上高,営業利益,経常利益,純利益,EPS,ROE,ROA
1109,4314,2010/12,104331000000,-1800000000,-24596000000,-2738000000,-1748.21,-,-
1169,4314,2011/12,33757000000,1470000000,-8769000000,6900000000,4405.64,-95.86,1.5
1206,4314,2012/12,14561000000,3541000000,-3558000000,1990000000,1270.61,-38.62,0.47
1229,4314,2013/12,202912000000,-31301000000,-44218000000,1701000000,1086.09,-50.53,1.21
1247,4314,2014/12,112735000000,1042000000,-48000000,6151000000,3927.41,216.28,63.03
1281,4314,2015/12,1475000000,121000000,-54000000,-2000000,-1.28,-,-
1318,4314,2016/12,7700000000,2141000000,1811000000,1792000000,1144.19,40.81,21.03
1340,4314,2017/12,1724000000,-224000000,-398000000,-257000000,-164.09,-,-
1364,4314,2018/12,1430000000,94000000,-300000000,-410000000,-261.78,-,-
1406,4314,2019/12,1049000000,-284000000,-621000000,-946000000,-604.02,-,-


### データ型の整合性確認
ロードしたファイルの方が各列で共通になるようにします。
<details>
<summary><b>結果</b></summary>
以下のように処理しています。

- コード：string
- 年度：timestump
- その他：float
</details>

In [ ]:
print("")
print("ファイル毎にデータを取得します")
df_DATAs_BY_ALL_FILEs = dl.load_data_by_files(
    settings["data_path"]["raw"],settings["years"],settings["files"]["raw"]
)

print("")
print("ハイフンを欠損値にします。")
df_missing_value_change = {
    (filename, year): df.replace("-", pd.NA)
    for (filename, year), df in df_DATAs_BY_ALL_FILEs.items()
}
display(df_missing_value_change)

print("")
print("数値化できるものはfloat化します。")
exclude_cols = ["コード", "年度"]  # 数値変換したくない列
def safe_to_numeric(x):
    try:
        return pd.to_numeric(x)
    except Exception:
        return x  # 数値変換できなければそのまま返す
df_type_to_float = {
    key: df.assign(
        **{
            col: df[col].apply(safe_to_numeric) 
            for col in df.columns if col not in exclude_cols
        }
    )
    for key, df in df_missing_value_change.items()
}
display(df_type_to_float)


In [ ]:
print("")
print("各列のdtypeと要素のtypeを出力します。")
def check_column_types(df_DATAs_BY_ALL_FILEs):
    COUNT_COLUMNS = ["filename", "year", "col", "dtype", "sample_types"]
    df_type = pd.DataFrame(columns=COUNT_COLUMNS)
    current_index = 0
    for (filename, year), df in df_DATAs_BY_ALL_FILEs.items():
        for col in df.columns:
            # 実際のデータ型を確認
            dtype = df[col].dtype
            sample_types = df[col].dropna().map(type).unique()
            row_data_tuple = (filename, year, col, dtype, sample_types)
            df_type.loc[current_index, COUNT_COLUMNS] = row_data_tuple
            #print(row_data_tuple)
            current_index +=1
    return df_type
df_type = check_column_types(df_type_to_float)
display(df_type)

print("")
print("dtypeがobjectの要素を検索します。")
display(df_type[df_type["dtype"] == "object"]["col"].unique())

print("")
print("dtypeがobjectの要素をstrにします。")
df = df_type_to_float
df_type_change = {
    #(filename, year): df.astype({col: 'string' for col in df.select_dtypes(include='object').columns})
    (filename, year): (df.assign(コード=df["コード"].astype("string")) if "コード" in df.columns else df)
    for (filename, year), df in df_type_to_float.items()
}
#display(df_type_change)
print("")
print("日付はdatetimeにします。")
df_type_change = {
    (filename, year): df.assign(年度=pd.to_datetime(df["年度"], format="%Y/%m"))
    for (filename, year), df in df_type_change.items()
}
print("")
print("変更されているか確認します。")
df_type = check_column_types(df_type_change)
#display(df_type[df_type["dtype"] == "object"]["col"].unique())
display(df_type)

print("")
print("最後にsample_typesが正しいか確認します")
def chk_sample_type(df_type):
    SAMPLE_TYPES_COLUMNS = ["col", "sample_types"]
    df_sample_types = pd.DataFrame(columns=SAMPLE_TYPES_COLUMNS)
    col_list = df_type["col"].unique().tolist()
    current_index=0
    for col in col_list:
        row_data_tuple = (
            col,
            df_type[df_type["col"]==col]["sample_types"].astype(str).unique()
        )
        df_sample_types.loc[current_index, SAMPLE_TYPES_COLUMNS] = row_data_tuple
        current_index +=1
    return df_sample_types
df_sample_types = chk_sample_type(df_type)
#display(df_sample_types)

print("")
print("float以外が含まれている列をfloatだけにします")
float_cols = df_sample_types[df_sample_types["sample_types"].astype(str).str.contains("int")]
float_cols = float_cols["col"].to_list()
df_type_change = {
    (filename, year): df.assign(**{
        col: df[col].astype(float)
        for col in float_cols
        if col in df.columns
    })
    for (filename, year), df in df_type_change.items()
}
df_type = check_column_types(df_type_change)
df_sample_types = chk_sample_type(df_type)
display(df_sample_types)

### 初期品質の確認

指標となる以下の数を確認し、必要な処理のプロセスを決定します。

- コード
- 年度
- 列

<summary><b>結果</b></summary>

</details>

In [ ]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_rows', 60)

print("")
print("処理済みデータをロードします。")
df_PROCESSED = df_type_change
#display(df_PROCESSED)

print("")
print("企業コード数、エンドの数、列数をファイルごとにまとめます。")
def get_file_info(df_PROCESSED):
    df_file_info = pd.DataFrame(columns=["file","year","code_counts","year_counts","column_counts"])
    for (filename, year), df in df_PROCESSED.items():
        # len(df_file_info)を次の新しい行のインデックスとして明示的に指定します。
        df_file_info.loc[len(df_file_info)] = [
            filename, year, df["コード"].nunique(), df["年度"].nunique(), df.columns.nunique()
        ]
    return df_file_info
df_file_info = get_file_info(df_PROCESSED)
#display(df_file_info)

print("")
print("各ファイルの年推移を可視化します。")
fig = px.line(df_file_info,x="year",y="code_counts",color="file")
#fig.show()
fig = px.line(df_file_info,x="year",y="year_counts",color="file")
#fig.show()
fig = px.line(df_file_info,x="year",y="column_counts",color="file")
#fig.show()

print("")
print("最新年のファイルに登録されている年度を表示します。")
df = df_PROCESSED[(settings["files"]["raw"][0],2025)]
#display(df["年度"].unique())

print("")
print("最新年に登録されている過去の年度と過去のデータが同じか確認します")
print("まずは、年度を限定して、コードの一覧を見ます")
#display(df[df["年度"]=="2024/12"]["コード"])

print("")
print("コードを限定し、過去のデータファイルと同じかどうか調べます。")
df = df_PROCESSED[(settings["files"]["raw"][0],2025)]
#display(df[df["コード"]=="130A"])
df = df_PROCESSED[(settings["files"]["raw"][0],2024)]
#display(df[df["コード"]=="130A"])

print("")
print("最新年度にある過去のデータを取得し、過去のデータを更新します")
def update_df(df_by_files, latest_year):
    df_all_datas_by_files = {}
    cutoff_date = cutoff_date = pd.to_datetime(f"{str(latest_year)}-01-01")
    #print(cutoff_date)
    all_files = {key[0] for key in df_by_files.keys()}
    for file in all_files:
        df_source_raw = df_by_files.get((file, latest_year))
        if df_source_raw is None:
            print(f"警告: {file} の2025年データが見つかりません。スキップします。")
            continue
        # フィルタリング条件を作成: '年度'が基準日より小さいデータのみ抽出
        df_update_source = df_source_raw[df_source_raw["年度"] < cutoff_date]
        df_update_source = df_update_source.set_index(["コード", "年度"])
        #display(df_update_source)
        # --- 2. 過去の全年のデータを更新 ---
        all_years = {key[1] for key in df_by_files.keys() if key[0] == file}
        for year in all_years:
            df_target = df_by_files.get((file, year))
            if df_target is None:
                continue
            df_target_indexed = df_target.set_index(["コード", "年度"])

            df_target_indexed.update(df_update_source)
            df_target_indexed = df_target_indexed.reset_index()
            df_all_datas_by_files[(file, year)] = df_target_indexed
    return df_all_datas_by_files

df_PROCESSED_AFTER = update_df(df_PROCESSED, 2025)

print("")
print("正しく処理が行われ、最新年に登録されている過去の年度と過去のデータが同じか確認します。")
df = df_PROCESSED_AFTER[(settings["files"]["raw"][0],2025)]
display(df[df["コード"]=="130A"])
df = df_PROCESSED_AFTER[(settings["files"]["raw"][0],2024)]
display(df[df["コード"]=="130A"])

print("")
print("最後に最新年度にある過去のデータを消去します。")
cutoff_date = pd.to_datetime("2025-01-01")
for file, year in df_PROCESSED_AFTER.keys():
    if year == 2025:
        df = df_PROCESSED_AFTER[(file,year)]
        df = df[df["年度"] >= cutoff_date]
        df_PROCESSED_AFTER[(file,year)] = df
print("")
print("各ファイルの年推移をもう一度可視化し、改善していることを確かめます。") 
df_file_info_after = get_file_info(df_PROCESSED_AFTER)
df_file_info_after["区分"] = "処理後"
df_file_info_after = df_file_info_after.set_index(["file", "year"]).sort_index()

df_file_info = get_file_info(df_PROCESSED)
df_file_info["区分"] = "処理前"
df_file_info = df_file_info.set_index(["file", "year"]).sort_index()
df_compare = pd.concat([df_file_info,df_file_info_after]).reset_index()
display(df_compare)
fig = px.line(df_compare, x="year", y="code_counts", color="file", line_dash="区分", markers=True)
fig.update_layout(title="処理前後のコード数推移", xaxis_title="年度", yaxis_title="コード数")
fig.show()

### 上場企業データのクリーニング

<summary><b>結果</b></summary>

</details>

In [ ]:
print("\nロードします。")
df_code_info = dl.load_on_startup(settings["data_path"]["reference"], "", settings["files"]["reference"],header=0)
#display(df_code_info)
dl.chk_file_missing(df_code_info)

print("\n欠損値表現を確認します。")
df_missing_values_expression = cu.chk_missing_values_expression(
    df_code_info, filename=settings["files"]["reference"], option_value=""
)
#display(df_missing_values_expression)

print("\n欠損値処理をします。")
df_code_info_missing_change = df_code_info.replace({"-": pd.NA, "": pd.NA})

print("\n数値列を Int64 に変換します。")
numeric_cols = ["17業種コード", "規模コード"]
df = df_code_info_missing_change
df = cu.convert_columns_type(df, numeric_cols, "int", True)

print("\nstring 型に統一します。")
df = df.assign(**{col: df[col].astype("string") for col in df.columns if col not in numeric_cols})

print("\n日付列を datetime に変換します。")
df["日付"] = pd.to_datetime(df["日付"], errors="coerce", format="%Y%m%d")
df_type_change = df

print("\n欠損値表現を確認します。")
df_missing_values_expression = cu.chk_missing_values_expression(df_type_change, filename=settings["files"]["reference"], option_value="")
display(df_missing_values_expression)

print("\nデータ型を確認します。")
df_type = cu.chk_dtype(df=df_type_change,filename=settings["files"]["reference"],option_value="",na_drop=True)
display(df_type)

display(df_type_change)